# Importing the libraries

In [17]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import AdamW
import torchvision
from torchvision.datasets import MNIST
import torchvision.transforms as tf
import numpy as np

In [14]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [16]:
print(device)

cpu


# Define the data transforms

In [12]:
train_tf = tf.Compose([tf.Resize((224, 224)),
                       tf.RandomHorizontalFlip(p=0.5),
                       tf.ToTensor(),
                       tf.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])

val_tf = tf.Compose([tf.Resize((224, 224)),
                    tf.ToTensor(),
                    tf.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])

# Get the MNIST dataset

In [15]:
train_dataset = MNIST(root='./data', train=True, download=True, transform=train_tf)
test_dataset = MNIST(root='./data', train=False, download=True, transform=val_tf)

Failed to download (trying next):
HTTP Error 404: Not Found



  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



# Get the dataloaders

In [18]:
trn_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, drop_last=True)
val_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, drop_last=True)

# Build the CNN model

In [20]:
model = nn.Sequential(nn.Conv2d(1, 64, kernel_size=3),
                      nn.ReLU(),
                      nn.MaxPool2d(2),
                      nn.Conv2d(64, 256, kernel_size=3),
                      nn.ReLU(),
                      nn.MaxPool2d(2),
                      nn.Conv2d(256, 128, kernel_size=3),
                      nn.ReLU(),
                      nn.MaxPool2d(2),
                      nn.Conv2d(128, 32, kernel_size=3),
                      nn.ReLU(),
                      nn.MaxPool2d(2),
                      nn.Flatten(),
                      nn.Linear(4608, 256),
                      nn.Linear(256, 128),
                      nn.Linear(128, 10))